In [529]:
import numpy as np
import pandas as pd
from IPython.display import display

In [530]:
df1 = pd.read_csv('../cnn2_output/benchmark/bm1.csv')
display(df1[:17])
df2 = pd.read_csv('../cnn2_output/benchmark/bm2.csv')
display(df2[:17])

,fname,label,prob
0,clip_000044442.wav,no,1.000000
1,clip_0000adecb.wav,unknown,1.000000
2,clip_0000d4322.wav,unknown,0.991082
3,clip_0000fb6fe.wav,up,0.297632
4,clip_0001d1559.wav,unknown,1.000000
5,clip_0002256ed.wav,unknown,1.000000
6,clip_0002a4a1f.wav,unknown,0.998950
7,clip_0002d9b83.wav,unknown,0.999977
8,clip_000373a5b.wav,go,0.999849
9,clip_0003c7122.wav,unknown,0.960456


,fname,label,prob
0,clip_000044442.wav,no,1.000000
1,clip_0000adecb.wav,unknown,1.000000
2,clip_0000d4322.wav,unknown,0.997975
3,clip_0000fb6fe.wav,up,0.657830
4,clip_0001d1559.wav,unknown,1.000000
5,clip_0002256ed.wav,unknown,0.999999
6,clip_0002a4a1f.wav,unknown,0.999985
7,clip_0002d9b83.wav,unknown,0.999992
8,clip_000373a5b.wav,go,1.000000
9,clip_0003c7122.wav,unknown,0.990678


In [531]:
df3 = pd.read_csv('../cnn2_output/benchmark/bm3.csv')
df4 = pd.read_csv('../cnn2_output/benchmark/bm4.csv')
df5 = pd.read_csv('../cnn2_output/benchmark/bm5.csv')

In [532]:
df0 = pd.read_csv('../cnn_output/predict.csv')
display(df0[:17])

,fname,label,prob
0,clip_000044442.wav,no,0.999997
1,clip_0000adecb.wav,happy,1.000000
2,clip_0000d4322.wav,four,0.909200
3,clip_0000fb6fe.wav,two,0.983358
4,clip_0001d1559.wav,tree,0.912538
5,clip_0002256ed.wav,three,0.999572
6,clip_0002a4a1f.wav,five,0.999285
7,clip_0002d9b83.wav,nine,0.999931
8,clip_000373a5b.wav,go,0.999906
9,clip_0003c7122.wav,eight,0.879851


In [533]:
(df1.fname.values != df2.fname.values).sum()

0

In [534]:
pred1 = df1.label.tolist()
pred2 = df2.label.tolist()
prob1 = df1.prob.tolist()
prob2 = df2.prob.tolist()
pred3 = df3.label.tolist()
pred4 = df4.label.tolist()
pred5 = df5.label.tolist()
prob3 = df3.prob.tolist()
prob4 = df4.prob.tolist()
prob5 = df5.prob.tolist()
pred0 = df0.label.tolist()
prob0 = df0.prob.tolist()

In [535]:
new_label = []
TAGET_LABELS = ['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go', 'silence', 'unknown']
hyper_ratio = 0.42
upper = 0.95
for i in range(len(df1)):
    if i % 30000 == 0:
        print i
    if pred1[i] == pred2[i]:
        new_label.append(pred1[i])
    elif prob1[i] > prob2[i] and prob1[i] > upper:
        new_label.append(pred1[i])
    elif prob2[i] > prob1[i] and prob2[i] > upper:
        new_label.append(pred2[i])
    else:
        cnt = {}
        cnt[pred1[i]] = prob1[i]**2
        cnt[pred2[i]] = prob2[i]**2
        if pred3[i] in cnt.keys():
            cnt[pred3[i]] += 0.08 * prob3[i]**2
        if pred4[i] in cnt.keys():
            cnt[pred4[i]] += 0.08 * prob4[i]**2
        if pred5[i] in cnt.keys():
            cnt[pred5[i]] += 0.08 * prob5[i]**2
        if pred0[i] not in TAGET_LABELS:
            pred0[i] = 'unknown'
        if pred0[i] in cnt.keys():
            cnt[pred0[i]] += 0.17 * prob0[i]**2
        leb = pred1[i]
        pro = prob1[i]**2
        for l in cnt.keys():
            if cnt[l] > pro:
                leb = l
                pro = cnt[l]
        if pro < hyper_ratio**2:
            leb = 'silence'
        new_label.append(leb)

0
30000
60000
90000
120000
150000


In [536]:
dic = {}
dic['fname'] = df1.fname.tolist()
dic['label'] = new_label
df_convert = pd.DataFrame(dic)
assert df_convert.fname.tolist() == df1.fname.tolist()
df_convert.set_index('fname', inplace = True)
display(df_convert[:15])

,label
fname,
clip_000044442.wav,no
clip_0000adecb.wav,unknown
clip_0000d4322.wav,unknown
clip_0000fb6fe.wav,up
clip_0001d1559.wav,unknown
clip_0002256ed.wav,unknown
clip_0002a4a1f.wav,unknown
clip_0002d9b83.wav,unknown
clip_000373a5b.wav,go


In [537]:
df_convert.to_csv('../cnn2_output/test.csv')
assert len(df_convert) == len(df1)

In [538]:
(df1.label.values != df2.label.values).sum()

9906

In [539]:
diff_files = []
for i in range(len(df1)):
    if pred1[i] != pred2[i]:
        diff_files.append(i)
for i in diff_files[:100]:
    print df1.fname.tolist()[i], pred1[i], prob1[i], pred2[i], prob2[i], pred3[i], pred4[i], pred5[i], pred0[i], new_label[i]

clip_0004c6707.wav go 0.176098540425 unknown 0.311636984348 unknown unknown unknown off silence
clip_00123c913.wav silence 0.180496811867 unknown 0.422477513552 go unknown unknown unknown unknown
clip_0021662c3.wav stop 0.441196024418 no 0.377702504396 unknown unknown unknown no stop
clip_0024bfdde.wav on 0.547341281493 unknown 0.485739498481 on on unknown unknown unknown
clip_0025e94de.wav right 0.762055397034 unknown 0.638788640499 right right right right right
clip_0027891d0.wav off 0.971619728611 silence 0.886538558396 silence silence silence silence off
clip_0038827a4.wav unknown 0.980549095902 left 0.516419649124 unknown unknown unknown up unknown
clip_00415ebb8.wav down 0.72006275893 no 0.614624798298 down down down down down
clip_00465ae81.wav up 0.566114008427 off 0.791240977719 up silence silence up off
clip_0051b16a9.wav off 0.422746951744 unknown 0.526660950581 unknown off off off unknown
clip_00560c89d.wav unknown 0.568423688412 on 0.5199086976 on unknown unknown on on
cli

In [540]:
test_path = '../data/test/audio/'
from scipy.io import wavfile
rate, samples = wavfile.read(test_path + '/' + 'clip_02b1dbeed.wav')
import IPython.display as ipd
ipd.Audio(samples, rate=rate)

In [541]:
for i in diff_files:
    
    

IndentationError: expected an indented block (<ipython-input-541-9dd83d4f4282>, line 3)